In [1]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Conv2D, Dense, BatchNormalization, MaxPooling2D, Dropout, Activation, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [2]:
Data_Dir= "DATA_DIR"
IMG_SIZE= 224
Batch = 32
EPOCHS = 25
CLASSES = ['Indian', 'USA', 'African', 'Other']

In [3]:
def conv_block(x,filters,kernel):
    x= Conv2D(filters, kernel, padding= "same")(x)
    x= BatchNormalization()(x)
    x= Activation("relu")(x)
    x= MaxPooling2D(2,2)(x)
    x= Dropout(0.25)(x)
    return x

In [4]:
def nationality_model(input_shape= (IMG_SIZE, IMG_SIZE), n_classes=4):
    inp= Input(shape=input_shape)
    x= conv_block(inp, 32, (3,3))
    x= conv_block(x, 64, (3,3))
    x= conv_block(x, 128, (3,3))
    x= Flatten()(x)
    x= Dense(256)(x)
    x= BatchNormalization()(x)
    x= Activation('relu')(x)
    x= Dropout(0.4)(x)
    out= Dense(n_classes,activation= "softmax")(x)
    model= Model(inputs= inp, outputs= out)
    model.compile(loss="categorical_crossentropy",optimizer= Adam(), metrics= ["accuracy"])
    
    return model

In [ ]:
def main():
    train_dir= os.path.join(Data_Dir, "train")
    val_dir= os.path.join(Data_Dir, "val")
    
    train_gen = ImageDataGenerator(
        rescale=1./255,
        rotation_range= 0,
        width_shift_range= 0.05,
        height_shift_range= 0.05,
        zoom_range = 0.1,
        horizontal_flip = True,
        fill_mode = "nearest"
    )

    val_gen = ImageDataGenerator(rescale=1./255)
    
    train_flow= train_gen.flow_from_directory(train_dir, target_size= (IMG_SIZE,IMG_SIZE), batch_size= Batch, classes= CLASSES, class_mode= "categorical")
    val_flow= val_gen.flow_from_directory(val_dir, target_size= (IMG_SIZE,IMG_SIZE), batch_size= Batch, classes= CLASSES, class_mode= "categorical")
    model= nationality_model((IMG_SIZE, IMG_SIZE, 3), len(CLASSES))
    model.summary()

    ckpt= ModelCheckpoint("nationalitymodel.weights.h5", monitor= "val_accuracy", save_weights_only= True, verbose=1)
    r1= ReduceLROnPlateau(monitor= "val_loss", factor= 0.5, verbose= 1)
    model.fit(train_flow, epochs= EPOCHS, validation_data= val_flow, callbacks= [ckpt,r1])
    print("Training finished. Best model saved to models/nationality_model.h5")

    model_json = model.to_json()
    with open('nationality_arch.json', 'w') as json_file:
        json_file.write(model_json)

    model.save('nationality_detection.h5')

if __name__ == "__main__":
   main()

Found 69394 images belonging to 4 classes.
Found 17350 images belonging to 4 classes.


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 224, 224, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_6 (Activation)       │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │    25,690,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_7 (Activation)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │         1,028 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,786,564 (98.37 MB)

 Trainable params: 25,785,604 (98.36 MB)

 Non-trainable params: 960 (3.75 KB)

C:\Users\HP\AppData\Roaming\Python\Python310\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
1116/2169 ━━━━━━━━━━━━━━━━━━━━ 2:28:37 8s/step - accuracy: 0.4828 - loss: 1.2960